In [1]:
import os
import time
import hashlib

from pathlib import Path
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

In [2]:
key = RSA.generate(2048)

In [3]:
# Save public key and private key
key_dir = './keys_RSA'
if not os.path.exists(key_dir):
        os.makedirs(key_dir)

with open(os.path.join(key_dir, 'public_key.txt'), 'wb') as f:
    f.write(key.publickey().export_key())

with open(os.path.join(key_dir, 'private_key.txt'), 'wb') as f:
    f.write(key.export_key())
    

In [4]:
def copy_folders(src_folder, dest_folder):
    for item in os.listdir(src_folder):
        if os.path.isdir(os.path.join(src_folder, item)):
            # Copy folders
            os.makedirs(os.path.join(dest_folder, item), exist_ok =  True)
            # Copy subfolders
            copy_folders(os.path.join(src_folder, item), os.path.join(dest_folder, item))

In [5]:
def encrypt_file(key_dir, data_dir):
    
    if not os.path.exists(key_dir):
        print('Cannot find keys')wdf
        
    else:
        with open(os.path.join(key_dir, 'public_key.txt'), 'rb') as f:
            public_key = RSA.import_key(f.read())
            
        j = 1
        BLOCK_SIZE = 128
        prev_dir_path = ''
        cipher = PKCS1_OAEP.new(public_key)

        filepaths = list(Path(data_dir).glob(r'**/*.flac'))
        print('Detected ' + str(len(filepaths)) + ' audio files in ' + os.path.abspath(data_dir))

        outfile_dir = data_dir + '_encrypted_RSA'
        if not os.path.exists(outfile_dir):
            os.makedirs(outfile_dir)
        print('Encrypted audio file is saved in', os.path.abspath(outfile_dir))
        
        copy_folders(data_dir, outfile_dir)
        
        start_time = time.time()

        for filename in filepaths:
            output_file = os.path.join(outfile_dir, os.path.splitext(str(filename))[0] + '_RSA.enc')
            output_file = output_file.replace(data_dir + '\\', '')

            dir_path = os.path.dirname(output_file)
            if dir_path != prev_dir_path:
                print('\nFor files in', os.path.abspath(dir_path))
                prev_dir_path = dir_path

            with open(str(filename), 'rb') as f:
                plaintext = f.read()
            ciphertext = b''
            for i in range(0, len(plaintext), BLOCK_SIZE):
                block = plaintext[i : i + BLOCK_SIZE]
                ciphertext += cipher.encrypt(block)
            with open(output_file, 'wb') as f:
                f.write(ciphertext)
            print(str(j) + ' ' + os.path.basename(str(filename)) + ' encrypted as ' + os.path.basename(output_file))
            j += 1
            
        end_time = time.time()
        
        print('\nTime for encrypting ' + str(len(filepaths)) 
              + ' files with RSA is %.2f' % (end_time - start_time) + 's')



def decrypt_file(key_dir, data_dir):
    
    if not os.path.exists(key_dir):
        print('Cannot find keys')
        
    else:
        with open(os.path.join(key_dir, 'private_key.txt'), 'rb') as f:
            private_key = RSA.import_key(f.read())
        
        j = 1
        prev_dir_path = ''
        BLOCK_SIZE = 128
        cipher = PKCS1_OAEP.new(private_key)

        filepaths = list(Path(data_dir).glob(r'**/*.enc'))
        print('Detected ' + str(len(filepaths)) + ' encrypted audio files in ' + os.path.abspath(data_dir))

        outfile_dir = './audio_decrypted_RSA'
        if not os.path.exists(outfile_dir):
            os.makedirs(outfile_dir)
        print('Decrypted audio file is saved in', os.path.abspath(outfile_dir))
        
        copy_folders(data_dir, outfile_dir)
        
        start_time = time.time()

        for filename in filepaths:
            output_file = os.path.join(outfile_dir, os.path.splitext(str(filename))[0] + '.flac')
            output_file = output_file.replace(data_dir + '\\', '')

            dir_path = os.path.dirname(output_file)
            if dir_path != prev_dir_path:
                print('\nFor files in', os.path.abspath(dir_path))
                prev_dir_path = dir_path
                
            with open(str(filename), 'rb') as f:
                ciphertext = f.read()
            plaintext = b''
            for i in range(0, len(ciphertext), private_key.size_in_bytes()):
                block = ciphertext[i : i + private_key.size_in_bytes()]
                plaintext += cipher.decrypt(block)
            with open(output_file, 'wb') as f:
                f.write(plaintext)
            print(str(j) + ' ' + os.path.basename(str(filename)) + ' decrypted as ' + os.path.basename(output_file))
            j += 1
        
        end_time = time.time()
        
        print('\nTime for decrypting ' + str(len(filepaths)) 
              + ' files with RSA is %.2f' % (end_time - start_time) + 's')

In [6]:
encrypt_file('keys_RSA', 'audio')

Detected 5323 audio files in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio
Encrypted audio file is saved in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_encrypted_RSA

For files in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_encrypted_RSA\dev-clean\1272\128104
1 1272-128104-0000.flac encrypted as 1272-128104-0000_RSA.enc
2 1272-128104-0001.flac encrypted as 1272-128104-0001_RSA.enc
3 1272-128104-0002.flac encrypted as 1272-128104-0002_RSA.enc
4 1272-128104-0003.flac encrypted as 1272-128104-0003_RSA.enc
5 1272-128104-0004.flac encrypted as 1272-128104-0004_RSA.enc
6 1272-128104-0005.flac encrypted as 1272-128104-0005_RSA.enc
7 1272-128104-0006.flac encrypted as 1272-128104-0006_RSA.enc
8 1272-128104-0007.flac encrypted as 1272-128104-0007_RSA.enc
9 1272-128104-0008.flac encrypted as 1272-128104-0008_RSA.enc
10 1272-128104-0009.flac encrypted as 1272-128104-0009_RSA.enc
11 1272-128104-0010.flac encrypted as 1272-128104-0010_RSA.enc
12 1272-128104-0011.flac encrypted as 1272-128104-00

121 1462-170142-0019.flac encrypted as 1462-170142-0019_RSA.enc
122 1462-170142-0020.flac encrypted as 1462-170142-0020_RSA.enc
123 1462-170142-0021.flac encrypted as 1462-170142-0021_RSA.enc
124 1462-170142-0022.flac encrypted as 1462-170142-0022_RSA.enc
125 1462-170142-0023.flac encrypted as 1462-170142-0023_RSA.enc
126 1462-170142-0024.flac encrypted as 1462-170142-0024_RSA.enc
127 1462-170142-0025.flac encrypted as 1462-170142-0025_RSA.enc
128 1462-170142-0026.flac encrypted as 1462-170142-0026_RSA.enc
129 1462-170142-0027.flac encrypted as 1462-170142-0027_RSA.enc
130 1462-170142-0028.flac encrypted as 1462-170142-0028_RSA.enc
131 1462-170142-0029.flac encrypted as 1462-170142-0029_RSA.enc
132 1462-170142-0030.flac encrypted as 1462-170142-0030_RSA.enc
133 1462-170142-0031.flac encrypted as 1462-170142-0031_RSA.enc
134 1462-170142-0032.flac encrypted as 1462-170142-0032_RSA.enc
135 1462-170142-0033.flac encrypted as 1462-170142-0033_RSA.enc
136 1462-170142-0034.flac encrypted as 1

243 174-50561-0010.flac encrypted as 174-50561-0010_RSA.enc
244 174-50561-0011.flac encrypted as 174-50561-0011_RSA.enc
245 174-50561-0012.flac encrypted as 174-50561-0012_RSA.enc
246 174-50561-0013.flac encrypted as 174-50561-0013_RSA.enc
247 174-50561-0014.flac encrypted as 174-50561-0014_RSA.enc
248 174-50561-0015.flac encrypted as 174-50561-0015_RSA.enc
249 174-50561-0016.flac encrypted as 174-50561-0016_RSA.enc
250 174-50561-0017.flac encrypted as 174-50561-0017_RSA.enc
251 174-50561-0018.flac encrypted as 174-50561-0018_RSA.enc
252 174-50561-0019.flac encrypted as 174-50561-0019_RSA.enc

For files in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_encrypted_RSA\dev-clean\174\84280
253 174-84280-0000.flac encrypted as 174-84280-0000_RSA.enc
254 174-84280-0001.flac encrypted as 174-84280-0001_RSA.enc
255 174-84280-0002.flac encrypted as 174-84280-0002_RSA.enc
256 174-84280-0003.flac encrypted as 174-84280-0003_RSA.enc
257 174-84280-0004.flac encrypted as 174-84280-0004_RSA.enc
258 17

367 1988-148538-0004.flac encrypted as 1988-148538-0004_RSA.enc
368 1988-148538-0005.flac encrypted as 1988-148538-0005_RSA.enc
369 1988-148538-0006.flac encrypted as 1988-148538-0006_RSA.enc
370 1988-148538-0007.flac encrypted as 1988-148538-0007_RSA.enc
371 1988-148538-0008.flac encrypted as 1988-148538-0008_RSA.enc
372 1988-148538-0009.flac encrypted as 1988-148538-0009_RSA.enc
373 1988-148538-0010.flac encrypted as 1988-148538-0010_RSA.enc
374 1988-148538-0011.flac encrypted as 1988-148538-0011_RSA.enc
375 1988-148538-0012.flac encrypted as 1988-148538-0012_RSA.enc
376 1988-148538-0013.flac encrypted as 1988-148538-0013_RSA.enc
377 1988-148538-0014.flac encrypted as 1988-148538-0014_RSA.enc
378 1988-148538-0015.flac encrypted as 1988-148538-0015_RSA.enc

For files in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_encrypted_RSA\dev-clean\1988\24833
379 1988-24833-0000.flac encrypted as 1988-24833-0000_RSA.enc
380 1988-24833-0001.flac encrypted as 1988-24833-0001_RSA.enc
381 1988-2483

487 2035-147961-0004.flac encrypted as 2035-147961-0004_RSA.enc
488 2035-147961-0005.flac encrypted as 2035-147961-0005_RSA.enc
489 2035-147961-0006.flac encrypted as 2035-147961-0006_RSA.enc
490 2035-147961-0007.flac encrypted as 2035-147961-0007_RSA.enc
491 2035-147961-0008.flac encrypted as 2035-147961-0008_RSA.enc
492 2035-147961-0009.flac encrypted as 2035-147961-0009_RSA.enc
493 2035-147961-0010.flac encrypted as 2035-147961-0010_RSA.enc
494 2035-147961-0011.flac encrypted as 2035-147961-0011_RSA.enc
495 2035-147961-0012.flac encrypted as 2035-147961-0012_RSA.enc
496 2035-147961-0013.flac encrypted as 2035-147961-0013_RSA.enc
497 2035-147961-0014.flac encrypted as 2035-147961-0014_RSA.enc
498 2035-147961-0015.flac encrypted as 2035-147961-0015_RSA.enc
499 2035-147961-0016.flac encrypted as 2035-147961-0016_RSA.enc
500 2035-147961-0017.flac encrypted as 2035-147961-0017_RSA.enc
501 2035-147961-0018.flac encrypted as 2035-147961-0018_RSA.enc
502 2035-147961-0019.flac encrypted as 2

...

4713 6930-81414-0008.flac encrypted as 6930-81414-0008_RSA.enc
4714 6930-81414-0009.flac encrypted as 6930-81414-0009_RSA.enc
4715 6930-81414-0010.flac encrypted as 6930-81414-0010_RSA.enc
4716 6930-81414-0011.flac encrypted as 6930-81414-0011_RSA.enc
4717 6930-81414-0012.flac encrypted as 6930-81414-0012_RSA.enc
4718 6930-81414-0013.flac encrypted as 6930-81414-0013_RSA.enc
4719 6930-81414-0014.flac encrypted as 6930-81414-0014_RSA.enc
4720 6930-81414-0015.flac encrypted as 6930-81414-0015_RSA.enc
4721 6930-81414-0016.flac encrypted as 6930-81414-0016_RSA.enc
4722 6930-81414-0017.flac encrypted as 6930-81414-0017_RSA.enc
4723 6930-81414-0018.flac encrypted as 6930-81414-0018_RSA.enc
4724 6930-81414-0019.flac encrypted as 6930-81414-0019_RSA.enc
4725 6930-81414-0020.flac encrypted as 6930-81414-0020_RSA.enc
4726 6930-81414-0021.flac encrypted as 6930-81414-0021_RSA.enc
4727 6930-81414-0022.flac encrypted as 6930-81414-0022_RSA.enc
4728 6930-81414-0023.flac encrypted as 6930-81414-0023_

4835 7127-75947-0013.flac encrypted as 7127-75947-0013_RSA.enc
4836 7127-75947-0014.flac encrypted as 7127-75947-0014_RSA.enc
4837 7127-75947-0015.flac encrypted as 7127-75947-0015_RSA.enc
4838 7127-75947-0016.flac encrypted as 7127-75947-0016_RSA.enc
4839 7127-75947-0017.flac encrypted as 7127-75947-0017_RSA.enc
4840 7127-75947-0018.flac encrypted as 7127-75947-0018_RSA.enc
4841 7127-75947-0019.flac encrypted as 7127-75947-0019_RSA.enc
4842 7127-75947-0020.flac encrypted as 7127-75947-0020_RSA.enc
4843 7127-75947-0021.flac encrypted as 7127-75947-0021_RSA.enc
4844 7127-75947-0022.flac encrypted as 7127-75947-0022_RSA.enc
4845 7127-75947-0023.flac encrypted as 7127-75947-0023_RSA.enc
4846 7127-75947-0024.flac encrypted as 7127-75947-0024_RSA.enc
4847 7127-75947-0025.flac encrypted as 7127-75947-0025_RSA.enc
4848 7127-75947-0026.flac encrypted as 7127-75947-0026_RSA.enc
4849 7127-75947-0027.flac encrypted as 7127-75947-0027_RSA.enc
4850 7127-75947-0028.flac encrypted as 7127-75947-0028_

4960 7729-102255-0023.flac encrypted as 7729-102255-0023_RSA.enc
4961 7729-102255-0024.flac encrypted as 7729-102255-0024_RSA.enc
4962 7729-102255-0025.flac encrypted as 7729-102255-0025_RSA.enc
4963 7729-102255-0026.flac encrypted as 7729-102255-0026_RSA.enc
4964 7729-102255-0027.flac encrypted as 7729-102255-0027_RSA.enc
4965 7729-102255-0028.flac encrypted as 7729-102255-0028_RSA.enc
4966 7729-102255-0029.flac encrypted as 7729-102255-0029_RSA.enc
4967 7729-102255-0030.flac encrypted as 7729-102255-0030_RSA.enc
4968 7729-102255-0031.flac encrypted as 7729-102255-0031_RSA.enc
4969 7729-102255-0032.flac encrypted as 7729-102255-0032_RSA.enc
4970 7729-102255-0033.flac encrypted as 7729-102255-0033_RSA.enc
4971 7729-102255-0034.flac encrypted as 7729-102255-0034_RSA.enc
4972 7729-102255-0035.flac encrypted as 7729-102255-0035_RSA.enc
4973 7729-102255-0036.flac encrypted as 7729-102255-0036_RSA.enc
4974 7729-102255-0037.flac encrypted as 7729-102255-0037_RSA.enc
4975 7729-102255-0038.fla

5081 8455-210777-0021.flac encrypted as 8455-210777-0021_RSA.enc
5082 8455-210777-0022.flac encrypted as 8455-210777-0022_RSA.enc
5083 8455-210777-0023.flac encrypted as 8455-210777-0023_RSA.enc
5084 8455-210777-0024.flac encrypted as 8455-210777-0024_RSA.enc
5085 8455-210777-0025.flac encrypted as 8455-210777-0025_RSA.enc
5086 8455-210777-0026.flac encrypted as 8455-210777-0026_RSA.enc
5087 8455-210777-0027.flac encrypted as 8455-210777-0027_RSA.enc
5088 8455-210777-0028.flac encrypted as 8455-210777-0028_RSA.enc
5089 8455-210777-0029.flac encrypted as 8455-210777-0029_RSA.enc
5090 8455-210777-0030.flac encrypted as 8455-210777-0030_RSA.enc
5091 8455-210777-0031.flac encrypted as 8455-210777-0031_RSA.enc
5092 8455-210777-0032.flac encrypted as 8455-210777-0032_RSA.enc
5093 8455-210777-0033.flac encrypted as 8455-210777-0033_RSA.enc
5094 8455-210777-0034.flac encrypted as 8455-210777-0034_RSA.enc
5095 8455-210777-0035.flac encrypted as 8455-210777-0035_RSA.enc
5096 8455-210777-0036.fla

5203 8463-294828-0037.flac encrypted as 8463-294828-0037_RSA.enc
5204 8463-294828-0038.flac encrypted as 8463-294828-0038_RSA.enc

For files in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_encrypted_RSA\test-clean\8555\284447
5205 8555-284447-0000.flac encrypted as 8555-284447-0000_RSA.enc
5206 8555-284447-0001.flac encrypted as 8555-284447-0001_RSA.enc
5207 8555-284447-0002.flac encrypted as 8555-284447-0002_RSA.enc
5208 8555-284447-0003.flac encrypted as 8555-284447-0003_RSA.enc
5209 8555-284447-0004.flac encrypted as 8555-284447-0004_RSA.enc
5210 8555-284447-0005.flac encrypted as 8555-284447-0005_RSA.enc
5211 8555-284447-0006.flac encrypted as 8555-284447-0006_RSA.enc
5212 8555-284447-0007.flac encrypted as 8555-284447-0007_RSA.enc
5213 8555-284447-0008.flac encrypted as 8555-284447-0008_RSA.enc
5214 8555-284447-0009.flac encrypted as 8555-284447-0009_RSA.enc
5215 8555-284447-0010.flac encrypted as 8555-284447-0010_RSA.enc
5216 8555-284447-0011.flac encrypted as 8555-284447-0011_R

In [7]:
decrypt_file('keys_RSA', 'audio_encrypted_RSA')

Detected 5323 encrypted audio files in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_encrypted_RSA
Decrypted audio file is saved in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_decrypted_RSA

For files in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_decrypted_RSA\dev-clean\1272\128104
1 1272-128104-0000_RSA.enc decrypted as 1272-128104-0000_RSA.flac
2 1272-128104-0001_RSA.enc decrypted as 1272-128104-0001_RSA.flac
3 1272-128104-0002_RSA.enc decrypted as 1272-128104-0002_RSA.flac
4 1272-128104-0003_RSA.enc decrypted as 1272-128104-0003_RSA.flac
5 1272-128104-0004_RSA.enc decrypted as 1272-128104-0004_RSA.flac
6 1272-128104-0005_RSA.enc decrypted as 1272-128104-0005_RSA.flac
7 1272-128104-0006_RSA.enc decrypted as 1272-128104-0006_RSA.flac
8 1272-128104-0007_RSA.enc decrypted as 1272-128104-0007_RSA.flac
9 1272-128104-0008_RSA.enc decrypted as 1272-128104-0008_RSA.flac
10 1272-128104-0009_RSA.enc decrypted as 1272-128104-0009_RSA.flac
11 1272-128104-0010_RSA.enc decrypted as 1272-12810

114 1462-170142-0012_RSA.enc decrypted as 1462-170142-0012_RSA.flac
115 1462-170142-0013_RSA.enc decrypted as 1462-170142-0013_RSA.flac
116 1462-170142-0014_RSA.enc decrypted as 1462-170142-0014_RSA.flac
117 1462-170142-0015_RSA.enc decrypted as 1462-170142-0015_RSA.flac
118 1462-170142-0016_RSA.enc decrypted as 1462-170142-0016_RSA.flac
119 1462-170142-0017_RSA.enc decrypted as 1462-170142-0017_RSA.flac
120 1462-170142-0018_RSA.enc decrypted as 1462-170142-0018_RSA.flac
121 1462-170142-0019_RSA.enc decrypted as 1462-170142-0019_RSA.flac
122 1462-170142-0020_RSA.enc decrypted as 1462-170142-0020_RSA.flac
123 1462-170142-0021_RSA.enc decrypted as 1462-170142-0021_RSA.flac
124 1462-170142-0022_RSA.enc decrypted as 1462-170142-0022_RSA.flac
125 1462-170142-0023_RSA.enc decrypted as 1462-170142-0023_RSA.flac
126 1462-170142-0024_RSA.enc decrypted as 1462-170142-0024_RSA.flac
127 1462-170142-0025_RSA.enc decrypted as 1462-170142-0025_RSA.flac
128 1462-170142-0026_RSA.enc decrypted as 1462-1

230 174-168635-0020_RSA.enc decrypted as 174-168635-0020_RSA.flac
231 174-168635-0021_RSA.enc decrypted as 174-168635-0021_RSA.flac
232 174-168635-0022_RSA.enc decrypted as 174-168635-0022_RSA.flac

For files in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_decrypted_RSA\dev-clean\174\50561
233 174-50561-0000_RSA.enc decrypted as 174-50561-0000_RSA.flac
234 174-50561-0001_RSA.enc decrypted as 174-50561-0001_RSA.flac
235 174-50561-0002_RSA.enc decrypted as 174-50561-0002_RSA.flac
236 174-50561-0003_RSA.enc decrypted as 174-50561-0003_RSA.flac
237 174-50561-0004_RSA.enc decrypted as 174-50561-0004_RSA.flac
238 174-50561-0005_RSA.enc decrypted as 174-50561-0005_RSA.flac
239 174-50561-0006_RSA.enc decrypted as 174-50561-0006_RSA.flac
240 174-50561-0007_RSA.enc decrypted as 174-50561-0007_RSA.flac
241 174-50561-0008_RSA.enc decrypted as 174-50561-0008_RSA.flac
242 174-50561-0009_RSA.enc decrypted as 174-50561-0009_RSA.flac
243 174-50561-0010_RSA.enc decrypted as 174-50561-0010_RSA.flac
244 

348 1988-147956-0015_RSA.enc decrypted as 1988-147956-0015_RSA.flac
349 1988-147956-0016_RSA.enc decrypted as 1988-147956-0016_RSA.flac
350 1988-147956-0017_RSA.enc decrypted as 1988-147956-0017_RSA.flac
351 1988-147956-0018_RSA.enc decrypted as 1988-147956-0018_RSA.flac
352 1988-147956-0019_RSA.enc decrypted as 1988-147956-0019_RSA.flac
353 1988-147956-0020_RSA.enc decrypted as 1988-147956-0020_RSA.flac
354 1988-147956-0021_RSA.enc decrypted as 1988-147956-0021_RSA.flac
355 1988-147956-0022_RSA.enc decrypted as 1988-147956-0022_RSA.flac
356 1988-147956-0023_RSA.enc decrypted as 1988-147956-0023_RSA.flac
357 1988-147956-0024_RSA.enc decrypted as 1988-147956-0024_RSA.flac
358 1988-147956-0025_RSA.enc decrypted as 1988-147956-0025_RSA.flac
359 1988-147956-0026_RSA.enc decrypted as 1988-147956-0026_RSA.flac
360 1988-147956-0027_RSA.enc decrypted as 1988-147956-0027_RSA.flac
361 1988-147956-0028_RSA.enc decrypted as 1988-147956-0028_RSA.flac
362 1988-147956-0029_RSA.enc decrypted as 1988-1

461 1993-147966-0002_RSA.enc decrypted as 1993-147966-0002_RSA.flac
462 1993-147966-0003_RSA.enc decrypted as 1993-147966-0003_RSA.flac
463 1993-147966-0004_RSA.enc decrypted as 1993-147966-0004_RSA.flac
464 1993-147966-0005_RSA.enc decrypted as 1993-147966-0005_RSA.flac
465 1993-147966-0006_RSA.enc decrypted as 1993-147966-0006_RSA.flac

For files in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_decrypted_RSA\dev-clean\2035\147960
466 2035-147960-0000_RSA.enc decrypted as 2035-147960-0000_RSA.flac
467 2035-147960-0001_RSA.enc decrypted as 2035-147960-0001_RSA.flac
468 2035-147960-0002_RSA.enc decrypted as 2035-147960-0002_RSA.flac
469 2035-147960-0003_RSA.enc decrypted as 2035-147960-0003_RSA.flac
470 2035-147960-0004_RSA.enc decrypted as 2035-147960-0004_RSA.flac
471 2035-147960-0005_RSA.enc decrypted as 2035-147960-0005_RSA.flac
472 2035-147960-0006_RSA.enc decrypted as 2035-147960-0006_RSA.flac
473 2035-147960-0007_RSA.enc decrypted as 2035-147960-0007_RSA.flac
474 2035-147960-0008

...

4786 7021-85628-0022_RSA.enc decrypted as 7021-85628-0022_RSA.flac
4787 7021-85628-0023_RSA.enc decrypted as 7021-85628-0023_RSA.flac
4788 7021-85628-0024_RSA.enc decrypted as 7021-85628-0024_RSA.flac
4789 7021-85628-0025_RSA.enc decrypted as 7021-85628-0025_RSA.flac
4790 7021-85628-0026_RSA.enc decrypted as 7021-85628-0026_RSA.flac
4791 7021-85628-0027_RSA.enc decrypted as 7021-85628-0027_RSA.flac

For files in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_decrypted_RSA\test-clean\7127\75946
4792 7127-75946-0000_RSA.enc decrypted as 7127-75946-0000_RSA.flac
4793 7127-75946-0001_RSA.enc decrypted as 7127-75946-0001_RSA.flac
4794 7127-75946-0002_RSA.enc decrypted as 7127-75946-0002_RSA.flac
4795 7127-75946-0003_RSA.enc decrypted as 7127-75946-0003_RSA.flac
4796 7127-75946-0004_RSA.enc decrypted as 7127-75946-0004_RSA.flac
4797 7127-75946-0005_RSA.enc decrypted as 7127-75946-0005_RSA.flac
4798 7127-75946-0006_RSA.enc decrypted as 7127-75946-0006_RSA.flac
4799 7127-75946-0007_RSA.enc decr

4903 7176-92135-0012_RSA.enc decrypted as 7176-92135-0012_RSA.flac
4904 7176-92135-0013_RSA.enc decrypted as 7176-92135-0013_RSA.flac
4905 7176-92135-0014_RSA.enc decrypted as 7176-92135-0014_RSA.flac
4906 7176-92135-0015_RSA.enc decrypted as 7176-92135-0015_RSA.flac
4907 7176-92135-0016_RSA.enc decrypted as 7176-92135-0016_RSA.flac
4908 7176-92135-0017_RSA.enc decrypted as 7176-92135-0017_RSA.flac
4909 7176-92135-0018_RSA.enc decrypted as 7176-92135-0018_RSA.flac
4910 7176-92135-0019_RSA.enc decrypted as 7176-92135-0019_RSA.flac
4911 7176-92135-0020_RSA.enc decrypted as 7176-92135-0020_RSA.flac
4912 7176-92135-0021_RSA.enc decrypted as 7176-92135-0021_RSA.flac
4913 7176-92135-0022_RSA.enc decrypted as 7176-92135-0022_RSA.flac
4914 7176-92135-0023_RSA.enc decrypted as 7176-92135-0023_RSA.flac
4915 7176-92135-0024_RSA.enc decrypted as 7176-92135-0024_RSA.flac
4916 7176-92135-0025_RSA.enc decrypted as 7176-92135-0025_RSA.flac
4917 7176-92135-0026_RSA.enc decrypted as 7176-92135-0026_RSA.

5018 8230-279154-0002_RSA.enc decrypted as 8230-279154-0002_RSA.flac
5019 8230-279154-0003_RSA.enc decrypted as 8230-279154-0003_RSA.flac
5020 8230-279154-0004_RSA.enc decrypted as 8230-279154-0004_RSA.flac
5021 8230-279154-0005_RSA.enc decrypted as 8230-279154-0005_RSA.flac
5022 8230-279154-0006_RSA.enc decrypted as 8230-279154-0006_RSA.flac
5023 8230-279154-0007_RSA.enc decrypted as 8230-279154-0007_RSA.flac
5024 8230-279154-0008_RSA.enc decrypted as 8230-279154-0008_RSA.flac
5025 8230-279154-0009_RSA.enc decrypted as 8230-279154-0009_RSA.flac
5026 8230-279154-0010_RSA.enc decrypted as 8230-279154-0010_RSA.flac
5027 8230-279154-0011_RSA.enc decrypted as 8230-279154-0011_RSA.flac
5028 8230-279154-0012_RSA.enc decrypted as 8230-279154-0012_RSA.flac
5029 8230-279154-0013_RSA.enc decrypted as 8230-279154-0013_RSA.flac
5030 8230-279154-0014_RSA.enc decrypted as 8230-279154-0014_RSA.flac
5031 8230-279154-0015_RSA.enc decrypted as 8230-279154-0015_RSA.flac
5032 8230-279154-0016_RSA.enc decr

5134 8463-287645-0003_RSA.enc decrypted as 8463-287645-0003_RSA.flac
5135 8463-287645-0004_RSA.enc decrypted as 8463-287645-0004_RSA.flac
5136 8463-287645-0005_RSA.enc decrypted as 8463-287645-0005_RSA.flac
5137 8463-287645-0006_RSA.enc decrypted as 8463-287645-0006_RSA.flac
5138 8463-287645-0007_RSA.enc decrypted as 8463-287645-0007_RSA.flac
5139 8463-287645-0008_RSA.enc decrypted as 8463-287645-0008_RSA.flac
5140 8463-287645-0009_RSA.enc decrypted as 8463-287645-0009_RSA.flac
5141 8463-287645-0010_RSA.enc decrypted as 8463-287645-0010_RSA.flac
5142 8463-287645-0011_RSA.enc decrypted as 8463-287645-0011_RSA.flac
5143 8463-287645-0012_RSA.enc decrypted as 8463-287645-0012_RSA.flac
5144 8463-287645-0013_RSA.enc decrypted as 8463-287645-0013_RSA.flac
5145 8463-287645-0014_RSA.enc decrypted as 8463-287645-0014_RSA.flac

For files in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_decrypted_RSA\test-clean\8463\294825
5146 8463-294825-0000_RSA.enc decrypted as 8463-294825-0000_RSA.flac
5147 8

5248 8555-284449-0018_RSA.enc decrypted as 8555-284449-0018_RSA.flac
5249 8555-284449-0019_RSA.enc decrypted as 8555-284449-0019_RSA.flac
5250 8555-284449-0020_RSA.enc decrypted as 8555-284449-0020_RSA.flac

For files in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_decrypted_RSA\test-clean\8555\292519
5251 8555-292519-0000_RSA.enc decrypted as 8555-292519-0000_RSA.flac
5252 8555-292519-0001_RSA.enc decrypted as 8555-292519-0001_RSA.flac
5253 8555-292519-0002_RSA.enc decrypted as 8555-292519-0002_RSA.flac
5254 8555-292519-0003_RSA.enc decrypted as 8555-292519-0003_RSA.flac
5255 8555-292519-0004_RSA.enc decrypted as 8555-292519-0004_RSA.flac
5256 8555-292519-0005_RSA.enc decrypted as 8555-292519-0005_RSA.flac
5257 8555-292519-0006_RSA.enc decrypted as 8555-292519-0006_RSA.flac
5258 8555-292519-0007_RSA.enc decrypted as 8555-292519-0007_RSA.flac
5259 8555-292519-0008_RSA.enc decrypted as 8555-292519-0008_RSA.flac
5260 8555-292519-0009_RSA.enc decrypted as 8555-292519-0009_RSA.flac
5261 8

In [8]:
def hash_file(data_dir):
    hash_dict = {}
    
    filepaths = list(Path(data_dir).glob(r'**/*.flac'))
    print('Detected ' + str(len(filepaths)) + ' audio files in ' + os.path.abspath(data_dir))
    
    for filename in filepaths:
            
        with open(filename, 'rb') as f:
            md5obj = hashlib.md5()
            md5obj.update(f.read())
            hash_value = md5obj.hexdigest()
            #print(os.path.basename(str(filename)) + ' hash value is ' + hash_value)
            
            if hash_value in hash_dict:
                hash_dict[hash_value].append(filename)
            else:
                hash_dict[hash_value] = [filename]
    
    return hash_dict

def compare_hash(data_dir1, data_dir2):
    i = 0
    filepaths1 = list(Path(data_dir1).glob(r'**/*.flac'))
    filepaths2 = list(Path(data_dir2).glob(r'**/*.flac'))
    
    hash_dict1 = hash_file(data_dir1)
    hash_dict2 = hash_file(data_dir2)
    
    for hash_value in hash_dict1:
        if hash_value in hash_dict2:
            i += 1
            print('\nFiles with hash value ' + hash_value + ' found in both directories:')
            print([os.path.abspath(str(f)) for f in hash_dict1[hash_value]][0])
            print([os.path.abspath(str(f)) for f in hash_dict2[hash_value]][0])
            
    print('\n{} ({:.2%}) files are the same after decrypted compared to original files'.format(i, i/len(filepaths1)))

In [9]:
compare_hash('audio', 'audio_decrypted_RSA')

Detected 5323 audio files in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio
Detected 5323 audio files in C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_decrypted_RSA

Files with hash value f9747a508acc1cffd47aa925d7985be8 found in both directories:
C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio\dev-clean\1272\128104\1272-128104-0000.flac
C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_decrypted_RSA\dev-clean\1272\128104\1272-128104-0000_RSA.flac

Files with hash value d559642f82dfe10b9da32c3533793811 found in both directories:
C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio\dev-clean\1272\128104\1272-128104-0001.flac
C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_decrypted_RSA\dev-clean\1272\128104\1272-128104-0001_RSA.flac

Files with hash value edecf45edbc50dc50633d7a42479488b found in both directories:
C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio\dev-clean\1272\128104\1272-128104-0002.flac
C:\Users\Ys1ong\ELEC0138\encryption\RSA\audio_decrypted_RSA\dev-clean\1272\128104\1272-128104-0002_RS